In [9]:
import numpy as np
import scipy.linalg as linalg
from scipy.stats import norm

import sys

# Define parent folder as relative path.
sys.path.append("python")

from ERADist import ERADist
from ERANataf import ERANataf
from ge_menendez_2017_traj_transformation import james_e_gentle_2006

In [ ]:
"""To DO: Write notebook for Lemaure Transfromation and its inverse. Also use non-standard normal distributions."""

In [17]:
row_approx = np.array([0.1, 0.1, 0.2, 0.8, 0.5])
# Transform uniform variable to standard normal space.
z = norm.ppf(row_approx) 
z.reshape(5,1)

array([[-1.28155157],
       [-1.28155157],
       [-0.84162123],
       [ 0.84162123],
       [ 0.        ]])

In [11]:
"""(Forward) Nataf Transformation"""

M = list()
M.append(ERADist('normal', 'PAR', [0, 1]))
M.append(ERADist('normal', 'PAR', [0, 1]))
M.append(ERADist('normal', 'PAR', [0, 1]))
M.append(ERADist('normal', 'PAR', [0, 1]))
M.append(ERADist('normal', 'PAR', [0, 1]))

cov = np.array([
        [1,0,0,0.2,0.5],
        [0,1,0.4,0.15,0],
        [0,0.4,1,0.05,0],
        [0.2,0.15,0.05,1,0],
        [0.5,0,0,0,1]])

# Correlation matrix.
Rho = cov

# Applying Nataf transformation
T_Nataf = ERANataf(M, Rho)

# Transform sample from INDEPENDENT standard normal to DEPENDENT actual/physical space.
X = T_Nataf.U2X(z)
X

array([[-1.28155157],
       [-1.28155157],
       [-1.28397923],
       [ 0.37548431],
       [-0.72770357]])

In [15]:
# Transform sample from uniform space to multivariate normal space, not necessarily independet or standard.
X_check = james_e_gentle_2006(row_approx, cov)
X_check.reshape(5,1)

array([[-1.28155157],
       [-1.28155157],
       [-1.28397923],
       [ 0.37548431],
       [-0.72770357]])

In [22]:
# Check wether inverse transformations lead to z.
# T_Nataf is not reversed because all parameters have the same distribution.
"""(Backward) Inverse Nataf Transformation"""
z_nataf = T_Nataf.X2U(X)

"""backward james e gentle"""
# Transform sample from DEPENDENT standard normal space to INDEPENDENT standard normal space.
M_prime = linalg.cholesky(cov, lower=True)
inv_M_prime = linalg.inv(M_prime)

indie_z = np.dot(inv_M_prime,X)
z_nataf.reshape(5,1)

array([[-1.28155157e+00],
       [-1.28155157e+00],
       [-8.41621234e-01],
       [ 8.41621234e-01],
       [-1.87422218e-16]])

In [23]:
indie_z

array([[-1.28155157e+00],
       [-1.28155157e+00],
       [-8.41621234e-01],
       [ 8.41621234e-01],
       [-2.15172689e-16]])

In [32]:
"""Show that reverse order of paramaters yields different results."""

rev_row_approx = np.array([0.5, 0.8, 0.2, 0.1, 0.1])

rev_cov = np.array([
        [1, 0, 0, 0, 0.5],
        [0, 1, 0.05, 0.15, 0.2],
        [0, 0.05, 1, 0.4, 0],
        [0, 0.15, 0.4, 1, 0],
        [0.5, 0.2, 0, 0, 1]])
rev_z = norm.ppf(rev_row_approx) 
rev_z.reshape(5,1)

array([[ 0.        ],
       [ 0.84162123],
       [-0.84162123],
       [-1.28155157],
       [-1.28155157]])

In [24]:


# Correlation matrix.
rev_Rho = rev_cov

# Applying Nataf transformation
T_Nataf = ERANataf(M, rev_Rho)


# Transform sample from INDEPENDENT standard normal to DEPENDENT actual/physical space.
rev_X = T_Nataf.U2X(rev_z)

# Transform sample from uniform space to multivariate normal space, not necessarily independet or standard.
rev_X_check = james_e_gentle_2006(rev_row_approx, rev_cov)

rev_X

array([[ 0.        ],
       [ 0.84162123],
       [-0.79848749],
       [-1.36716278],
       [-0.86557956]])

In [26]:
rev_X_check.reshape(5,1)

array([[ 0.        ],
       [ 0.84162123],
       [-0.79848749],
       [-1.36716278],
       [-0.86557956]])

In [27]:
"""
Result: The order matters! The first element stays untouched. Also the previous paramters
influence the following but not vice versa!!!
--> The "Rosenblatt Transformation is not unique for dependent parameters.
It depends on the ordering of marginal (conditional) cdfs.
"""

'\nResult: The order matters! The first element stays untouched. Also the previous paramters\ninfluence the following but not vice versa!!!\n--> The "Rosenblatt Transformation is not unique.\n'

In [31]:
# Check again
"""(Backward) james e gentle (to standard normal!)"""
# Transform sample from DEPENDENT standard normal space to INDEPENDENT standard normal space.
rev_M_prime = linalg.cholesky(rev_cov, lower=True)
rev_inv_M_prime = linalg.inv(rev_M_prime)

rev_indie_z = np.dot(rev_inv_M_prime, rev_X)


"""(Backward) Inverse Nataf Transformation (to standard normal!)"""
rev_z_nataf = T_Nataf.X2U(rev_X)
rev_z_nataf.reshape(5,1)

array([[ 0.        ],
       [ 0.84162123],
       [-0.84162123],
       [-1.28155157],
       [-1.28155157]])

In [30]:
rev_indie_z

array([[ 0.        ],
       [ 0.84162123],
       [-0.84162123],
       [-1.28155157],
       [-1.28155157]])